In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import imageio
from pathlib import Path


In [2]:
stress = pd.read_csv('raw_data/data_stress/ASI_AnnualSummary_Season1_data.csv')

Data Source: https://www.fao.org/giews/earthobservation/country/index.jsp?lang=en&type=1141&code=ARM#

In [3]:
# Pivot -> provinces are rows, years are columns, values are stress
# Useful for inspection and renaming
pivot_df = stress.pivot_table(index='PROVINCE', columns='YEAR', values='DATA', aggfunc='mean')

pivot_df = pivot_df.sort_index().sort_index(axis=1)

pivot_df.columns = pivot_df.columns.astype(int)

# rename specific index entries for merge in R file
pivot_df = pivot_df.rename(index={
    'Vayots Dzor': 'Vayots dzor',
    'Lori': 'Lor',
    'Gergharkunik' : 'Gegharkunik'
})

# convert pivoted table back to long/tidy format for R
long_df = pivot_df.reset_index().melt(id_vars='PROVINCE', var_name='YEAR', value_name='DATA')
long_df['YEAR'] = long_df['YEAR'].astype(int)
long_df

,PROVINCE,YEAR,DATA
0,ALL,1984,0.03
1,Aragatsotn,1984,0.21
2,Ararat,1984,0.00
3,Armavir,1984,0.18
4,Gegharkunik,1984,0.00
...,...,...,...
487,Shirak,2024,0.00
488,Syunik,2024,0.09
489,Tavush,2024,1.69
490,Vayots dzor,2024,0.00


In [4]:
#Save like this for relative paths

notebook_dir = Path().resolve()
save_dir = notebook_dir.parent / "drought_data_output"
save_dir.mkdir(exist_ok=True)
file_path = save_dir / "agri_stress_long.csv"
file_path
long_df.to_csv(file_path, index=False)


In [5]:
# import armenia shapefile
gdf = gpd.read_file(notebook_dir.parent / 'armenia_shapefile/am.json')

In [6]:
# map index names in pivot_df so they match the shapefile 'name' values (adjust mapping as needed)
mapping = {
    'Gergharkunik': 'Gegharkunik',
    'Yerevan': 'Erevan',
    'Vayots dzor': 'Vayots Dzor',
    'Lor': 'Lori',
    # add more mappings here if you find other mismatches
}


pivot_df = pivot_df.rename(index=mapping)

# update reset version used for merging
pivot_reset = pivot_df.reset_index()

# quick checks
print("Current pivot_df index:", list(pivot_df.index))
print("Names in pivot not in gdf:", sorted(set(pivot_df.index) - set(gdf['name'])))

Current pivot_df index: ['ALL', 'Aragatsotn', 'Ararat', 'Armavir', 'Gegharkunik', 'Kotayk', 'Lori', 'Shirak', 'Syunik', 'Tavush', 'Vayots Dzor', 'Erevan']
Names in pivot not in gdf: ['ALL']


In [8]:
# save the pivoted DataFrame to CSV
pivot_df.to_csv(notebook_dir.parent / 'drought_data_output/agri_stress_pivoted.csv')

# Visual Representation
This first cell commented out is just printing every single image

In [9]:

# Read the Armenia GeoJSON/shapefile (uses am.json if available)
gpath_json = notebook_dir.parent / 'armenia_shapefile/am.json'
gdf = gpd.read_file(gpath_json)

In [10]:
frames_dir = 'frames/frames_stress'
os.makedirs(frames_dir, exist_ok=True)
filenames = []

# Prepare pivot table for merging: reset index so province name is a column
pivot_reset = pivot_df.reset_index()
idx_name = 'PROVINCE'

# colormap and fixed range 0..100 -> consistency across frames
cmap = plt.get_cmap('viridis')
vmin, vmax = 0, 100

# ensure years sorted
years = list(pivot_df.columns)
years = sorted(years)

for year in years:
    df_year = pivot_reset[[idx_name, year]].rename(columns={year: 'value'})
    merged = gdf.merge(df_year, left_on='name', right_on=idx_name, how='left')

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 8))
    merged.plot(column='value', ax=ax, cmap=cmap, vmin=vmin, vmax=vmax,
                edgecolor='black', linewidth=0.5, legend=True,
                missing_kwds={'color': 'lightgrey', 'label': 'no data'})
    ax.set_title(f'Annual agricultural stress — {year}')
    ax.axis('off')

    fname = os.path.join(frames_dir, f'frame_{year}.png')
    fig.savefig(fname, bbox_inches='tight', dpi=150)
    plt.close(fig)
    filenames.append(fname)

# build gif by essentially adding images together
images = [imageio.imread(f) for f in filenames]
imageio.mimsave('gifs/agri_stress.gif', images, duration=0.8)

C:\Users\jonas\AppData\Local\Temp\ipykernel_2912\1476372571.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(f) for f in filenames]
